<a href="https://colab.research.google.com/github/almedida/thesis/blob/main/simulation_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
rm(list = ls())

##**Install Packages**##

In [10]:
install.packages('pacman')
library(pacman, devtools)
p_load("tidyverse", "matrixTests", "gtools", "VennDiagram", "tmvtnorm")

#install BiocManager
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

#install limma and qvalue packages
BiocManager::install(c("limma", "qvalue"))
library(limma)
library(qvalue)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘matrixStats’



matrixTests installed

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


gtools installed

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘formatR’, ‘lambda.r’, ‘futile.options’, ‘futile.logger’



VennDiagram installed

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘zoo’, ‘sandwich’, ‘mvtnorm’, ‘gmm’



tmvtnorm installed

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


Bioconductor version 3.14 (BiocManager 1.3

##***Load Dataset***

In [11]:
#load dataset
simulation_data = read.table("sim_data.csv", header= TRUE, sep=",")


N = ncol(simulation_data) #N = number columns (samples in the dataset)
n_gene = nrow(simulation_data) #n_gene = no of genes (rows in the dataset)

In [ ]:
# head(simulation_data)

In [ ]:
# dim(simulation_data)

10k randomly selected data for simulation process

In [12]:
#set.seed(1)
m = 10000 #no of genes to be selected from the dataset
randomly_selected_genes = sample(n_gene, m) #randomly selected genes of size m from the total genes

# head(randomly_selected_genes)
# length(randomly_selected_genes)

In [13]:
#10000 selected data corresponding to the randomly selected genes (EE genes)
randomly_selected_data = simulation_data[randomly_selected_genes, ]
genej_sim_data = randomly_selected_data
head(genej_sim_data)
# dim(genej_sim_data)

,GSM1247203,GSM1247204,GSM1247205,GSM1247206,GSM1247207,GSM1247208,GSM1247209,GSM1247210,GSM1247211,GSM1247212,⋯,GSM1247410,GSM1247411,GSM1247412,GSM1247413,GSM1247414,GSM1247415,GSM1247416,GSM1247417,GSM1247418,GSM1247419
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2573,5.696752,5.732106,5.527625,5.765523,5.794510,5.673148,5.661434,5.792428,5.902410,5.850823,⋯,5.683829,5.738701,5.729288,5.629334,5.815015,5.606970,5.733598,5.786336,5.786699,5.635683
20153,7.958196,8.240848,8.108602,7.962404,7.850494,7.661667,8.036861,8.050551,8.311008,7.968991,⋯,8.109547,8.137065,8.067874,8.141278,8.124685,7.813756,7.912128,7.885002,8.000819,7.939448
9349,6.573637,6.722796,6.621485,6.680845,6.735268,6.565933,6.670599,6.571579,6.659489,6.881373,⋯,6.629452,6.560661,6.811317,6.545243,6.229667,6.541565,6.883910,6.669711,6.828555,6.774223
509,10.494920,10.068500,10.421450,11.074220,9.751422,10.354780,10.289660,10.229210,10.489280,9.790936,⋯,10.624440,10.273820,9.645807,10.839760,10.407380,10.291670,10.828140,10.228550,9.595561,10.189640
1084,4.475790,4.306536,4.353308,4.277311,5.876474,4.479235,4.167353,3.935195,4.094814,4.109920,⋯,4.294724,4.483528,4.363303,4.313966,4.006860,4.249210,4.366151,3.779328,4.329857,4.648503
5129,7.459041,7.370674,8.240806,8.094577,7.351286,7.901789,7.420691,8.119202,8.188850,7.591702,⋯,7.863736,7.967049,7.839592,7.685373,7.920784,8.162983,7.551414,7.964921,7.252925,8.204767


standarad deviation for 10000 genes from all experimental units

In [14]:
genej_std = apply(genej_sim_data, 1, sd)
# head(genej_std)
# length(genej_std)

## **Simulation**

Here, we create 2 treatment groups with 4 randomly selected samples each with 10,000 randlomly selected genes.

In [15]:
set.seed(Sys.time())
n_samples = 4 #no of samples
randomly_selected_samples  = sample(N, 2*n_samples) #randomly select samples of size 2*n_trt_grp 
                                                    #for treatment group
treatment_group1 = randomly_selected_samples[1:n_samples]
treatment_group2 = randomly_selected_samples[(n_samples + 1):(2*n_samples)]

Group 1 data

In [16]:
treatment_group1_data = genej_sim_data[treatment_group1]
# head(treatment_group1_data)
# write.csv(treatment_group1_data, "10k_untreated_group15.csv")

* Group 1 consists of EE genes and group 2 consists of 9k EE genes and 1k DE genes (genes with treatment effects)
* treatment_group2_data consists of the 4 randomly selected samples (exp. units)

In [17]:
treatment_group2_data = genej_sim_data[treatment_group2]
# dim(treatment_group2_data)
head(treatment_group2_data, 2)

,GSM1247300,GSM1247261,GSM1247275,GSM1247322
,<dbl>,<dbl>,<dbl>,<dbl>
2573,5.716673,5.570183,5.878891,5.813386
20153,8.231208,7.829603,8.222807,8.120285


**Simulation from 3.1**

We calculate the treatment effect by extracting the effect size of each gene j from a random normal distribution using genej_std (standard deviation for the 10k randomly selected genes from all experimental units in the population data)

In [18]:
set.seed(1)
mu = genej_std

treatment_effect_j = rnorm(10000, mu, genej_std)
# length(treatment_effect_j)
#head(data.frame(treatment_effect_j))


**DE genes**

DE genes are created by adding treatment effects extracted to each gene in group 2

In [19]:
dim(treatment_group2_data)
de_treatment_group2 = treatment_group2_data + treatment_effect_j
# head(de_treatment_group2)

[1] 10000     4

## we randomly selected 3000 genes with treatment effect from de_treatment_group2

In [20]:
# set.seed(1)
n_de = 1000 
n_treated_genes_grp2 = nrow(de_treatment_group2)

#we extract 3000 genes from DE_treatment_group2 (genes with treatment effects)
de_genes_index = sample(n_treated_genes_grp2, n_de)

#we copied untreated (genes without effects) genes from treatment_group2_data
treated_genes_group2 = treatment_group2_data

#we replaced the 3000 untreated genes (without treatment effects) with the corresponding treated genes(with treatment effects)
treated_genes_group2[de_genes_index, ] = de_treatment_group2[de_genes_index, ]

# dim(treated_genes_group2)
# head(de_genes_index)

## **Summary of Group 2 simulation** 

m00 - number of EE genes

m11 - number of DE genes


In [21]:
m00 = treated_genes_group2[-de_genes_index, ]
m11 = treated_genes_group2[de_genes_index, ]
treatment_effect_size_grp2 = treatment_effect_j[de_genes_index]
# write.csv(treated_genes_group2, "10k_treated_group25.csv")
#dim(m00)
#dim(m11)
# length(treatment_effect_size_grp2)

## **Student t-test**

In [22]:
group1_data = treatment_group1_data
group2_data = treated_genes_group2

In [23]:
ttest <- row_t_equalvar(group1_data, group2_data, alternative = "two.sided", mu = 0, conf.level = 0.95)

In [24]:
pvalue <- ttest[13]
head(pvalue)
# hist_data <- data.matrix(pvalue)
# hist(hist_data)

,pvalue
,<dbl>
2573,6.096962e-01
20153,5.834692e-01
9349,1.738878e-01
509,3.798330e-06
1084,1.652852e-01
5129,8.526334e-02


**Adjusted p-values using Benjamini Hochberg**

In [25]:
adj_pvalue <- p.adjust(data.matrix(pvalue), "BH")
adj_pvalue_df <-  data.frame(adj_pvalue)

## **Limma t-test**

In [26]:
limma_gene_exp <- c(group1_data, group2_data)
limma_gene_exp <- data.frame(limma_gene_exp)

**Create expression list**

create a factor list for the differentially expressed genes with group1 set as first level

In [27]:
expr_list_10k <- factor(
  x = c(rep("group1",4), rep("group2",4)),
  levels=c("group1","group2")            # Set group 1 to be the first level
)

design <- model.matrix(~expr_list_10k)          # Remove the zero
# head(design)

In [28]:
# #Now we can run the differential expression pipeline

fit <- lmFit(limma_gene_exp, design)
fit <- eBayes(fit)
results <- decideTests(fit)
# write.csv(summary(results), "summary_10k_genes_expr_sim5.csv")

In [30]:
head(fit$p.value)

(Intercept),expr_list_10kgroup2
3.177115e-13,6.535316e-01
1.157367e-13,5.778525e-01
4.293840e-13,1.632579e-01
2.278502e-15,3.185251e-07
4.432785e-07,9.015440e-02
8.139978e-14,8.267542e-02


In [29]:
limma_pvalue <- fit$p.value[, 2]
limma_pvalue <- data.frame(limma_pvalue)
head(limma_pvalue)

,limma_pvalue
,<dbl>
1,6.535316e-01
2,5.778525e-01
3,1.632579e-01
4,3.185251e-07
5,9.015440e-02
6,8.267542e-02


In [ ]:
adj_limma_pvalue <- data.matrix(limma_pvalue)
adj_limma_pvalue <- p.adjust(adj_limma_pvalue, "BH")
adj_limma_pvalue <- data.frame(adj_limma_pvalue)

## **Proposed Method**

In [ ]:
pval_raw <- c(pvalue, limma_pvalue)
pval_raw <- data.frame(pval_raw)
pvals1 <- (as.matrix(pval_raw[,1]))
pvals2 <- (as.matrix(pval_raw[,2]))

**Calculate cut-off points using Histogram-based method**

calc.cutoff slightly modifies the estimate.m0 function to return the appropriate p-value cutoff where all p-values greater than the cutoff are assumed to come from null cases.

In [ ]:
calc.cutoff = function(p, B = 20, max=1){

  m <- length(p)
  m0 <- m
  bin <- c(-0.1, (1:B)/B*max)
  bin.counts=rep(0,B)

  for(i in 1:B){
    bin.counts[i]=sum((p>bin[i])&(p<=bin[i+1]))
  }

  tail.means <- rev(cumsum(rev(bin.counts))/(1:B))
  temp <- bin.counts - tail.means
  index <- min((1:B)[temp <= 0])
  cutoff2 <- (index)/B*max
  if(cutoff2 == 1) {cutoff2 <- 1-1/B}

  return(cutoff2)

}

cutoff_value1 = calc.cutoff(pvals1, B=20, max=1)
cutoff_value2 = calc.cutoff(pvals2, B=20, max=1)

cutoff = cbind(c(cutoff_value1), c(cutoff_value2))

colnames(cutoff) = c("cutoff_value1", "cutoff_value2")

cutoff

cutoff_value1,cutoff_value2
0.85,0.85


**Filter dataset using cutoff points**

selecting pvalues in the upper-right quadrant using Histogram-Based method
defined as lambda1 = cutoffvalue1 and lambda2 = cutoffvalue2 for pvalue 1 and pvalue 2 respectively.
lambda1, lambda2 - probability of pvalue 1 and pvalue 2 in the upper-right quadrant respectively

In [ ]:
p_vals = pval_raw  %>% filter(pvalue >=cutoff_value1, limma_pvalue>=cutoff_value2)

convert selected pvalues to z values

In [ ]:
z_val = as.data.frame(qnorm(as.matrix(p_vals), lower.tail = TRUE))
colnames(z_val) = c("zvals1", "zvals2")

zvals1 <- (as.data.frame(as.matrix(z_val[,1])))
zvals2 <- (as.data.frame(as.matrix(z_val[,2])))

convert lambda(truncation points) to z values

In [ ]:
z_val_extremums = as.data.frame(qnorm(as.matrix(cbind(c(cutoff_value1,1),c(cutoff_value2,1))), lower.tail = TRUE))

min_z1 <- z_val_extremums[1,1]
min_z2 <- z_val_extremums[1,2]

**Estimating m00s**

m0.1 - DE genes in experiment 1
m0.2 - DE genes in experiment 2
m11 - DE genes in both experiments
m00 - EE genes in both experiments

In [ ]:
estimate.m0s <- function(p1, p2, B=20){
  m <- length(p1)

  ##find lambda cutoffs using histogram-based method
  c1 <- calc.cutoff(p1, B=B, max=1)
  c2 <- calc.cutoff(p2, B=B, max=1)

  ##estimate m0 for experiment 1
  ind1 <- (p1>=c1)
  m0.1 <- sum(ind1)/(1-c1)
  m0.1 <- min(m0.1, 10000)

  ##estimate m0 for experiment 2  
  ind2 <- (p2>=c2)
  m0.2 <- sum(ind2)/(1-c2)
  m0.2 <- min(m0.2, 10000)


  ##estimate m00
  ind12 <- ind1 & ind2
  nA <- sum(ind12)
  #pA <- (1-c1)*(1-c2)
  #m00 <- nA/pA
  
  #here, we used converted pvalues to z values to estimnate m00
  # density function for each row of the bivariate z values (x) and 
  # estimated parameters(rho)
  density = function(x, rho)
  {
    sigma = matrix(c(1, rho, rho, 1), 2, 2)
    z = dtmvnorm(x, mean = c(0,0), sigma = sigma, lower = c(min_z1, min_z2))
  }
  
  # log likelihood of the joint densities
  log_likelihood_fn = function(rho){
    
    joint_likelihood = z_val %>% split(.$zvals2) %>% map_dfr(~density(c(.$zvals1,.$zvals2),rho))    
    return(-sum(log(joint_likelihood)))
    
  }
  
  #MLE of the log likelihood function
  optimal_rho = optimize(log_likelihood_fn, lower = -1, upper = 1 )
  optimal_rho = as.data.frame(optimal_rho)
  
  #probability of a random variable greater than cutoff values 
  rho = as.numeric(optimal_rho[1])
  obj_value = optimal_rho[2]
  pA = pmvnorm(lower=c(min_z1, min_z2), upper=c(Inf, Inf), mean=c(0,0), sigma = matrix(c(1, rho, rho, 1), 2, 2))
  

  m00 <- nA/pA
  m00 <- min(m00, 10000)
  
  
  ##estimate m11
  m11 <- sum(m - m0.1 - m0.2 + m00)
  if (m0.1 == 10000 || m0.2 == 10000 || m00 == 10000){
    m11 = 0
  }
  
  ret <- list()
  ret$ms <- c(obj_value, rho, m, m0.1, m0.2, m00, m11)
  names(ret$ms) <- c("obj_value", "optimal_rho", "m", "m0.1", "m0.2", "m00", "m11")
  ret$cutoffs <- c(c1, c2)
  return(ret)
}

estimate.m0s(pvals1, pvals2, B=20)

$ms
$ms$obj_value
[1] -488.5183

$ms$optimal_rho
[1] 0.996325

$ms$m
[1] 10000

$ms$m0.1
[1] 4693.333

$ms$m0.2
[1] 4706.667

$ms$m00
[1] 4654.091

$ms$m11
[1] 5254.091


$cutoffs
[1] 0.85 0.85

# **Orr, 2012 Method**

In [ ]:
estimate.m0s <- function(p1, p2, B=20){
  m <- length(p1)

  ##find lambda cutoffs using histogram-based method
  c1 <- calc.cutoff(p1, B=B, max=1)
  c2 <- calc.cutoff(p2, B=B, max=1)

  ##estimate m0 for experiment 1
  ind1 <- (p1>=c1)
  m0.1 <- sum(ind1)/(1-c1)
  m0.1 <- min(m0.1, 10000)
  

  ##estimate m0 for experiment 2  
  ind2 <- (p2>=c2)
  m0.2 <- sum(ind2)/(1-c2)
  m0.2 <- min(m0.2, 10000)

  ##estimate m00
  ind12 <- ind1 & ind2
  nA <- sum(ind12)
  pA <- (1-c1)*(1-c2)
  m00 <- nA/pA
  m00 <- min(m00, 10000)


  ##estimate m11
  m11 <- sum(m - m0.1 - m0.2 + m00)
  if (m00 == 10000 || m0.1 == 10000 || m0.2 == 10000){
    m11 = 0
  }
  
  ret <- list()
  ret$ms <- c(m, m0.1, m0.2, m11, m00)
  names(ret$ms) <- c("m", "m0.1", "m0.2", "m11","m00")
  ret$cutoffs <- c(c1, c2)
  return(ret)
}

estimate.m0s(pvals1, pvals2, B=20)

$ms
        m      m0.1      m0.2       m11       m00 
10000.000  4693.333  4706.667     0.000 10000.000 

$cutoffs
[1] 0.85 0.85

# **Intersection Method**

In [ ]:
adj_pvalue_df <- data.frame(adj_pvalue)
# head(adj_pvalue_df)

In [ ]:
pvalue_p <- adj_pvalue_df[rowSums(adj_pvalue_df[1]<=0.05), ]
pvalue_05 <- length(pvalue_p)
#pvalue_05

In [ ]:
limma_p <- adj_limma_pvalue[rowSums(adj_limma_pvalue[1]<=0.05), ]
limma_05 <- length(limma_p) 
#limma_05

In [ ]:
limma_ttest <- c(adj_pvalue_df, adj_limma_pvalue)
limma_ttest <- data.frame(limma_ttest)

In [ ]:
limma_ttest_p <- limma_ttest[rowSums((limma_ttest[1]<=0.05) & (limma_ttest[2]<=0.05)), ]
limma_ttest_05 <- nrow(limma_ttest_p)
limma_ttest_05

[1] 1352

In [ ]:
pvalue_p2 <- adj_pvalue_df[rowSums(adj_pvalue_df[1]<=0.1), ]
pvalue_1 <- length(pvalue_p2)
#pvalue_1

In [ ]:
limma_p2 <- adj_limma_pvalue[rowSums(adj_limma_pvalue[1]<=0.1), ]
limma_1 <- length(limma_p2) 
#limma_1

In [ ]:
# write.csv(limma_ttest, "adj_pvalue_limma.csv", row.names=FALSE)

In [ ]:
limma_ttest_p2 <- limma_ttest[rowSums((limma_ttest[1]<=0.1) & (limma_ttest[2]<=0.1)), ]
limma_ttest_1 <- nrow(limma_ttest_p2)
limma_ttest_1

[1] 2207

# **Venn diagram Analysis**

Controlling at alpha<=0.05 using Benjamini-Hochberg

In [ ]:
# draw.pairwise.venn(area1=limma_1, area2=pvalue_1,cross.area=limma_ttest_1,
#                    category=c("Student t-test","Limma"),fill=c("Red","Yellow"),
# )